In [1]:
import json
from pprint import pprint

import redis
from tqdm.auto import tqdm

db = redis.Redis(host="localhost", port=6379, db=1)

In [2]:
dataset = json.load(open("../data/logs_with_position.json"))

In [3]:
all_docs = {}
missing_docs = set()
for u in tqdm(dataset):
    for x in u["clicks"]:

        url = x["url"]
        if url in all_docs or url in missing_docs:
            continue
        if not db.exists(url):
            pprint(f"missed doc {url}")
            missing_docs.add(url)
            continue
        doc_text = db.get(url)

        all_docs[url] = doc_text

  0%|          | 0/127 [00:00<?, ?it/s]

'missed doc https://www.aboutibs.org/ibs-diet.html'
('missed doc '
 'https://www.differencebetween.com/difference-between-metaethics-and-normative-ethics/')
'missed doc http://www.c14dating.com/frac.html'
'missed doc http://www.c14dating.com/pret.html'
'missed doc https://hmo-ealing.uk/united-states-subprime-mortgage-crisis/'
'missed doc https://en.scio.pw/Subprime_mortgage_crisis'
'missed doc https://enacademic.com/dic.nsf/enwiki/11272120'
('missed doc '
 'https://irritable-bowel-syndrome-forum.com/irritable-bowel-syndrome-treatment-wikipedia/')
'missed doc https://www.wikiyy.com/en/Subprime_mortgage_crisis'
'missed doc https://megsmenopause.com/2020/01/29/ibs/'
('missed doc '
 'https://www.managedcaremag.com/archives/2016/2/economic-evaluation-linaclotide-treatment-adult-patients-chronic-idiopathic')
'missed doc https://www.teachengineering.org/lessons/view/uoh_genetic_lesson01'
('missed doc '
 'https://knowgenetics.org/introduction-to-genetically-modified-organisms-gmos/')
('missed 

In [6]:
with open("../data/missing_docs.txt", "w") as outf:
    for u in missing_docs:
        outf.write(f"{u}\n")

In [22]:
with open("../data/clicked_docs.tsv", "w") as outf:
    for (k, v) in tqdm(all_docs.items()):
        text = v.decode()
        clean_text = text.replace("[SEPREDIS]", " ").replace("\n", " ").replace("\t", " ")
        outf.write(f"{k}\t{clean_text}\n")

  0%|          | 0/1081 [00:00<?, ?it/s]